<a href="https://colab.research.google.com/github/Shopping-Yuan/ML2021HW/blob/Shopping_branch/HW1/ML2021HW1_modified.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

pytorch 設置:
torch.backends.cudnn 設定捲積的優化方式，無捲積層故關閉。

In [ ]:
#import pytorch and set CNN algorithm
import torch
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False
def get_device():
    return 'cuda' if torch.cuda.is_available() else 'cpu'
device = get_device()
print(torch.cuda.is_available())
#set random variable
import numpy as np
myseed = 1
np.random.seed(myseed)
torch.manual_seed(myseed)
if torch.cuda.is_available():
    torch.cuda.manual_seed_all(myseed)

True


In [ ]:
#rewrite class Dataset
import torch
import pandas as pd
from torch.utils.data import Dataset, DataLoader

class my_Dataset(Dataset):
  def __init__(self,name,mode,data_set_function):
    self.mode = mode
    self.name = name
    if self.mode == 'test':
      self.data = data_set_function()
    else:
      self.data = data_set_function(mode)["data"]
      self.label = data_set_function(mode)["label"]

    self.dim = self.data.shape[0]
    print('Finished reading the {mode} set of {name} Dataset ({len} samples found, each dim = {dim})'
            .format(mode = self.mode, name = self.name, len =len(self.data), dim=self.dim))

  def __getitem__(self, index):
      if self.mode in ['train', 'val']:
          return self.data[index], self.label[index]
      else:
          return self.data[index]

  def __len__(self):
      return len(self.data)

In [40]:
#write data_set_function
train_path = "./covid.train.csv"
test_path = "./covid.test.csv"
def normalize(df):
  return ((df - df.mean(axis = 0)) / (df.std(axis = 0)))
def got_index(df):
  train_index,dev_index = [],[]
  first_t_data = 0
  for i in range(df.shape[0]):
    days = df[i].sum()
    last_t_data = df[i].sum()
def covid19_train_dev_f(mode):
    train_df = pd.read_csv(train_path)
    print('Size of training data: {}'.format(train_df.shape))

    train_df.iloc[:,41:] = normalize(train_df.iloc[:,41:])

    train_index , dev_index = got_index(train_df[1:41])

    if mode == 'train':
      train_df = train_df[ lambda x : x.index in train_index]
    elif mode == 'dev':
      train_df = train_df[ lambda x : x.index in dev_index]
    label = torch.FloatTensor(train_df.iloc[:, -1].to_numpy())
    data = torch.FloatTensor(train_df.iloc[:, 0:-1].to_numpy())
    return({"data":data , "label":label})
def covid19_test_f():
    test_df = pd.read_csv(test_path)
    print('Size of testing data: {}'.format(test_df.shape))
    test_df.iloc[:,40:] = normalize(test_df.iloc[:,40:])
    data = torch.FloatTensor(test_df.to_numpy())
    return(data)


#decide how to load data
def prep_dataloader(path, mode, batch_size, name , data_set_function , n_jobs=0,):
    dataset = my_Dataset(path, name ,mode,data_set_function)
    dataloader = DataLoader(
        dataset, batch_size,
        shuffle=(mode == 'train'), drop_last=False,
        num_workers=n_jobs, pin_memory=True)
    return dataloader

In [ ]:
# import pandas as pd
# df = pd.DataFrame([[1,2],[3,4]])
# print(df)
# print(df.mean(axis = 0))
# print(df - df.mean(axis = 0))
# print(df - df.std(axis = 0))
# print((df - df.mean(axis = 0)) / (df.std(axis = 0)))
# df.iloc[:,1] = normalize(df.iloc[:,1])
# print(df)

In [ ]:
#train = pd.read_csv(train_path)
# train.iloc[:,40:] = normalize(train.iloc[:,40:])
# #print(train.shape,train.iloc[:,40:])
# train = train.iloc[ lambda x : (x+1).index % 67 <= 60]
# print(train.shape,train.iloc[:,40:])
# print(torch.FloatTensor(train.to_numpy()))
#print(covid19_train_dev_f("train"))

Size of training data: (2700, 95)
{'data': tensor([[ 0.0000e+00,  1.0000e+00,  0.0000e+00,  ..., -9.3829e-01,
         -1.7432e+00, -1.8429e-01],
        [ 1.0000e+00,  1.0000e+00,  0.0000e+00,  ..., -1.0103e+00,
         -1.7120e+00, -3.6366e-01],
        [ 2.0000e+00,  1.0000e+00,  0.0000e+00,  ..., -1.0966e+00,
         -1.8002e+00, -3.0505e-01],
        ...,
        [ 2.6970e+03,  0.0000e+00,  0.0000e+00,  ..., -1.9965e-01,
          4.6573e-01, -1.1139e+00],
        [ 2.6980e+03,  0.0000e+00,  0.0000e+00,  ..., -4.0093e-02,
          4.7601e-01, -1.1417e+00],
        [ 2.6990e+03,  0.0000e+00,  0.0000e+00,  ...,  1.9403e-02,
          5.5462e-01, -1.1687e+00]]), 'label': tensor([ 0.5609,  0.6381,  0.6215,  ..., -0.5011, -0.5189, -0.6063])}


In [ ]:
# train_set = my_Dataset("Covid19","train",covid19_train_dev_f)
# print(type(train_set),train_set[0][0].size())

Size of training data: (2700, 95)
Size of training data: (2700, 95)
Finished reading the train set of Covid19 Dataset (2460 samples found, each dim = 2460)
<class '__main__.my_Dataset'> torch.Size([94])


In [46]:
# df = pd.read_csv(train_path)
# df_onehot = train.iloc[:,1:41]
# print(df_onehot.head())
# df["state"] = pd.from_dummies(df_onehot)
# print(df.iloc[:,[1,-1]].head())

    AL state
0  1.0    AL
1  1.0    AL
2  1.0    AL
3  1.0    AL
4  1.0    AL


In [ ]:
from torch.nn.modules.activation import Sigmoid
import torch.nn as nn
class Multiply100Layer(nn.Module):
    def __init__(self):
        super(Multiply100Layer, self).__init__()

    def forward(self, x):
        return 100.0*x
class NeuralNet(nn.Module):
    ''' A simple fully-connected deep neural network '''
    def __init__(self, input_dim):
        super(NeuralNet, self).__init__()

        # Define your neural network here
        # TODO: How to modify this model to achieve better performance?
        model = nn.Sequential(
            nn.Linear(input_dim, 64),
            nn.ReLU(),
            nn.Linear(64, 1),
            nn.Sigmoid(),
            Multiply100Layer(),
        )

        self.criterion = nn.MSELoss(reduction='mean')

    def forward(self, x):
        return model(x)

    def cal_loss(self, pred, target):

        return self.criterion(pred, target)

In [ ]:
def train(tr_set, dv_set, model, config, device):
    ''' DNN training '''

    n_epochs = config['n_epochs']  # Maximum number of epochs

    # Setup optimizer
    optimizer = getattr(torch.optim, config['optimizer'])(
        model.parameters(), **config['optim_hparas'])

    min_mse = 1000.
    loss_record = {'train': [], 'dev': []}      # for recording training loss
    early_stop_cnt = 0
    epoch = 0
    while epoch < n_epochs:
        model.train()                           # set model to training mode
        for x, y in tr_set:                     # iterate through the dataloader
            optimizer.zero_grad()               # set gradient to zero
            x, y = x.to(device), y.to(device)   # move data to device (cpu/cuda)
            pred = model(x)                     # forward pass (compute output)
            mse_loss = model.cal_loss(pred, y)  # compute loss
            mse_loss.backward()                 # compute gradient (backpropagation)
            optimizer.step()                    # update model with optimizer
            loss_record['train'].append(mse_loss.detach().cpu().item())

        # After each epoch, test your model on the validation (development) set.
        dev_mse = dev(dv_set, model, device)
        if dev_mse < min_mse:
            # Save model if your model improved
            min_mse = dev_mse
            print('Saving model (epoch = {:4d}, loss = {:.4f})'
                .format(epoch + 1, min_mse))
            torch.save(model.state_dict(), config['save_path'])  # Save model to specified path
            early_stop_cnt = 0
        else:
            early_stop_cnt += 1

        epoch += 1
        loss_record['dev'].append(dev_mse)
        if early_stop_cnt > config['early_stop']:
            # Stop training if your model stops improving for "config['early_stop']" epochs.
            break

    print('Finished training after {} epochs'.format(epoch))
    return min_mse, loss_record

In [ ]:
def dev(dv_set, model, device):
    model.eval()                                # set model to evalutation mode
    total_loss = 0
    for x, y in dv_set:                         # iterate through the dataloader
        x, y = x.to(device), y.to(device)       # move data to device (cpu/cuda)
        with torch.no_grad():                   # disable gradient calculation
            pred = model(x)                     # forward pass (compute output)
            mse_loss = model.cal_loss(pred, y)  # compute loss
        total_loss += mse_loss.detach().cpu().item() * len(x)  # accumulate loss
    total_loss = total_loss / len(dv_set.dataset)              # compute averaged loss

    return total_loss

In [ ]:
def test(tt_set, model, device):
    model.eval()                                # set model to evalutation mode
    preds = []
    for x in tt_set:                            # iterate through the dataloader
        x = x.to(device)                        # move data to device (cpu/cuda)
        with torch.no_grad():                   # disable gradient calculation
            pred = model(x)                     # forward pass (compute output)
            preds.append(pred.detach().cpu())   # collect prediction
    preds = torch.cat(preds, dim=0).numpy()     # concatenate all predictions and convert to a numpy array
    return preds

In [ ]:
tr_set = prep_dataloader(tr_path, 'train', config['batch_size'], target_only=target_only)
dv_set = prep_dataloader(tr_path, 'dev', config['batch_size'], target_only=target_only)
tt_set = prep_dataloader(tt_path, 'test', config['batch_size'], target_only=target_only)

In [ ]:
device = get_device()                 # get the current available device ('cpu' or 'cuda')
os.makedirs('models', exist_ok=True)  # The trained model will be saved to ./models/
target_only = False                   # TODO: Using 40 states & 2 tested_positive features

# TODO: How to tune these hyper-parameters to improve your model's performance?
config = {
    'n_epochs': 3000,                # maximum number of epochs
    'batch_size': 270,               # mini-batch size for dataloader
    'optimizer': 'SGD',              # optimization algorithm (optimizer in torch.optim)
    'optim_hparas': {                # hyper-parameters for the optimizer (depends on which optimizer you are using)
        'lr': 0.001,                 # learning rate of SGD
        'momentum': 0.9              # momentum for SGD
    },
    'early_stop': 200,               # early stopping epochs (the number epochs since your model's last improvement)
    'save_path': 'models/model.pth'  # your model will be saved here
}

In [ ]:
model = NeuralNet(tr_set.dataset.dim).to(device)  # Construct model and move to device
model_loss, model_loss_record = train(tr_set, dv_set, model, config, device)
del model
model = NeuralNet(tr_set.dataset.dim).to(device)
ckpt = torch.load(config['save_path'], map_location='cpu')  # Load your best model
model.load_state_dict(ckpt)

In [ ]:
def save_pred(preds, file):
    ''' Save predictions to specified file '''
    print('Saving results to {}'.format(file))
    with open(file, 'w') as fp:
        writer = csv.writer(fp)
        writer.writerow(['id', 'tested_positive'])
        for i, p in enumerate(preds):
            writer.writerow([i, p])

preds = test(tt_set, model, device)  # predict COVID-19 cases with your model
save_pred(preds, 'pred.csv')         # save prediction file to pred.csv